# Set up

In [1]:
import duckdb
import pandas as pd
from IPython.display import display
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from eda_support_functions import *

# Suppress all warnings
import warnings

warnings.filterwarnings('ignore')

In [2]:
full__regression__none = pd.read_csv('../data/ml_ready_data/full__regression__with_outliers__none.csv')

In [3]:
full__regression__complex = pd.read_csv('../data/ml_ready_data/full__regression__with_outliers__complex.csv')

In [4]:
full__regression__complex.shape

(12131, 387)

In [5]:
full__regression__complex = full__regression__complex.convert_dtypes(infer_objects=True)

In [6]:
# Make a dataframe with the columns that we have in the full__regression__complex dataframe, and then a column with the splits ("__") in the column names
full__regression__complex_columns = pd.DataFrame(full__regression__complex.columns, columns=['column_name'])
full__regression__complex_columns['level_1'] = full__regression__complex_columns['column_name'].apply(lambda x: x.split('__')[0])
full__regression__complex_columns['level_2'] = full__regression__complex_columns['column_name'].apply(lambda x: x.split('__')[1] if len(x.split('__')) > 1 else None)

In [7]:
full__regression__complex_columns['boolean_true'] = full__regression__complex_columns['column_name'].apply(lambda x: full__regression__complex[x].sum() if full__regression__complex[x].dtype == 'boolean' else None)

In [8]:
a = full__regression__complex_columns[full__regression__complex_columns['boolean_true'].notnull()].sort_values('boolean_true', ascending=False)
a['normalized'] = a['boolean_true'] / full__regression__complex.shape[0]

In [9]:
a

,column_name,level_1,level_2,boolean_true,normalized
310,is_spoken_language__en,is_spoken_language,en,9082.0,0.748660
7,is_released__US,is_released,US,8483.0,0.699283
373,is_prod_country__US,is_prod_country,US,7382.0,0.608524
58,is_genre__Drama,is_genre,Drama,6439.0,0.530789
9,is_released__FR,is_released,FR,6367.0,0.524854
...,...,...,...,...,...
260,is_collection__los_superagentes,is_collection,los_superagentes,0.0,0.000000
259,is_collection__lone_wolf_and_cub_collection,is_collection,lone_wolf_and_cub_collection,0.0,0.000000
258,is_collection__les_charlots_saga,is_collection,les_charlots_saga,0.0,0.000000
256,is_collection__lemon_popsicle,is_collection,lemon_popsicle,0.0,0.000000


In [10]:
a.groupby('level_1').agg({
    'normalized': ['mean', 'min', 'max']
}).rename(columns={'normalized': ''})

mean       min       max
level_1                                           
is_collection         0.000191  0.000000  0.001484
is_genre              0.124201  0.000659  0.530789
is_keyword            0.019269  0.001566  0.077735
is_on_holiday_window  0.403347  0.403347  0.403347
is_outlier            0.455527  0.455527  0.455527
is_prod_company       0.007705  0.000000  0.052840
is_prod_country       0.032704  0.000495  0.608524
is_released           0.391279  0.038084  0.699283
is_spoken_language    0.047092  0.001896  0.748660

In [11]:
full__regression__complex_columns.groupby('level_1').agg({'level_2': 'count', 'boolean_true': 'sum'}).sort_values('level_2', ascending=False)

,level_2,boolean_true
level_1,,
is_collection,91,211.0
is_prod_company,89,8319.0
is_keyword,53,12389.0
is_prod_country,42,16663.0
is_spoken_language,29,16567.0
is_genre,19,28627.0
is_released,5,23733.0
actor_kpis,4,0.0
producer_kpis,4,0.0


In [12]:
full__regression__none

,movie_id,original_language,runtime,ageCert,quarter,month,year,is_released__US,is_released__CN,is_released__FR,is_released__GB,is_released__JP,budget_usd_adj,revenue_usd_adj,is_outlier,production_size
0,7551,en,126,PG13,4,11,2006,True,True,True,True,True,1.133564e+08,2.728980e+08,False,large_productions
1,7553,en,94,R,4,10,2005,True,False,False,True,False,4.680522e+06,2.907804e+07,False,small_productions
2,23830,en,93,U,4,11,2010,False,False,True,True,False,9.781496e+06,1.068270e+07,True,small_productions
3,66150,en,95,PG13,1,1,2011,True,False,False,False,False,6.123055e+04,3.059509e+05,False,small_productions
4,505058,en,93,R,3,7,2018,True,False,True,True,False,1.213435e+06,1.941496e+07,True,small_productions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12126,23827,en,86,R,3,9,2007,True,False,True,True,False,3.159559e+05,2.841484e+08,True,small_productions
12127,441881,hi,136,U,2,6,2017,False,False,False,True,False,1.923815e+07,4.061942e+07,True,medium_productions
12128,7520,en,104,R,3,7,1988,True,False,True,True,True,5.151344e+07,4.417401e+08,True,large_productions
12129,505015,es,87,U,1,3,2018,False,False,False,False,False,4.732396e+06,5.096427e+06,True,small_productions


In [13]:
full__regression__none.shape

(12131, 16)

In [14]:
(full__regression__none
 .groupby(['is_outlier', 'production_size'])
 .size()
 .unstack(fill_value=0)
 .rename(index={False: 'Not Outlier', True: 'Outlier'})
 .assign(Total=lambda x: x.sum(axis=1))
 .pipe(lambda df: pd.concat([df, pd.DataFrame([df.sum()], index=['Total'])])
       .fillna(0)
       .astype(int))
 .style
#  .background_gradient(cmap='YlOrRd')
 .format("{:,d}")
)

production_size,large_productions,medium_productions,small_productions,Total
Not Outlier,"3,471","1,362","1,772","6,605"
Outlier,797,"1,040","3,689","5,526"
Total,"4,268","2,402","5,461","12,131"


In [15]:
full__regression__none['ratio'] = full__regression__none['budget_usd_adj'] / full__regression__none['revenue_usd_adj']

In [18]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=False)[['movie_id','original_language','year','ratio', 'budget_usd_adj','revenue_usd_adj']].head(30).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,original_language,year,ratio,budget_usd_adj,revenue_usd_adj
318,24285,nl,2003,10.530729,"$6,905,475","$655,745"
2062,253120,en,1985,10.526316,"$2,831,803","$269,021"
10264,230266,en,2014,10.434571,"$7,079,029","$678,421"
10488,4228,fr,1989,10.416667,"$122,863,710","$11,794,916"
123,8046,en,2003,10.404325,"$125,192,778","$12,032,763"
4857,12653,en,2004,10.400512,"$7,258,650","$697,913"
6177,33799,fr,1998,10.381921,"$17,384,838","$1,674,530"
10813,21052,en,2003,10.373444,"$12,419,918","$1,197,280"
10981,89325,en,2012,10.369348,"$15,925,608","$1,535,835"
258,24200,en,2006,10.359098,"$3,022,837","$291,805"


In [17]:
display(full__regression__none[~full__regression__none['is_outlier']].sort_values(by='ratio', ascending=True)[['movie_id', 'ratio', 'budget_usd_adj','revenue_usd_adj']].head(10).style.format({'budget_usd_adj': "${:,.0f}",'revenue_usd_adj': "${:,.0f}"}))

,movie_id,ratio,budget_usd_adj,revenue_usd_adj
9931,122906,0.137772,"$15,695,703","$113,925,235"
8149,82695,0.137811,"$80,955,173","$587,437,688"
11284,70,0.138400,"$48,391,001","$349,646,990"
7712,609,0.138544,"$33,785,610","$243,862,610"
1380,9522,0.138648,"$62,406,964","$450,110,225"
7418,402,0.138839,"$106,417,662","$766,483,471"
6587,277216,0.138865,"$35,995,967","$259,215,942"
6860,297802,0.138885,"$194,149,586","$1,397,911,470"
5562,109513,0.138889,"$2,654,268","$19,110,729"
3466,11077,0.138889,"$23,313,083","$167,854,200"
